In [1]:
%pylab inline
%reload_ext autoreload
%autoreload 2

from __future__ import division
from __future__ import print_function

import os
import copy
import argparse
import fnmatch
import numpy as np

import scipy
from scipy.interpolate import interp1d

# Astropy
from astropy.io import fits
from astropy    import units as u
from astropy.stats import sigma_clip
# AstroML
from astroML.plotting import hist
# Astropy.Table
from astropy.table import Table, Column, MaskedColumn, TableColumns

# Matplotlib related
# Matplotlib default settings
rcdef = plt.rcParams.copy()
pylab.rcParams['figure.figsize'] = 12, 10
pylab.rcParams['xtick.major.size'] = 8.0
pylab.rcParams['xtick.major.width'] = 2.5
pylab.rcParams['xtick.minor.size'] = 4.0
pylab.rcParams['xtick.minor.width'] = 2.5
pylab.rcParams['ytick.major.size'] = 8.0
pylab.rcParams['ytick.major.width'] = 2.5
pylab.rcParams['ytick.minor.size'] = 4.0
pylab.rcParams['ytick.minor.width'] = 2.5
pylab.rcParams['legend.numpoints'] = 1

import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
from matplotlib.ticker import NullFormatter
from matplotlib.ticker import MaxNLocator

Populating the interactive namespace from numpy and matplotlib


In [2]:
# Read in the data
catDir = '/Users/songhuang/work/hscs/gama_compare/redmapper'
catMem = os.path.join(catDir, 'hsc_redmapper_mem_wide15a_2arcsec_match_flux_cmodel.fits')
memData = Table.read(catMem, format='fits')

In [3]:
# Number of galaxies
len(memData)

17385

In [4]:
# ID of unique clusters
clusters = np.unique(memData['ID_CLUSTER'])

In [5]:
# Isolate those BCGs
bcgs = np.where(memData['OBJID_MEM'] == memData['OBJID_BCG'])
bcgs[0].shape

(265,)

In [16]:
# Remove the BCGs from the table 
memNew = copy.deepcopy(memData)
memNew.remove_rows(bcgs)
len(memNew)

17120

In [17]:
# New columns for the outputs
newID = np.chararray(len(memNew), itemsize=20)
newP = (memNew['P_CEN_1']) * 0.0

In [18]:
# Add two new columns for New ID and P_CEN value 
memNew.add_column(Column(name='ID_MEM', data=newID))
memNew.add_column(Column(name='P_CEN_MEM', data=newP))

In [26]:
# Update the P_CEN_MEM
memNew['P_CEN_MEM'][memNew['OBJID_MEM'] == memNew['ID_CEN_2']] = memNew[memNew['OBJID_MEM'] == 
                                                                        memNew['ID_CEN_2']]['P_CEN_2']
memNew['P_CEN_MEM'][memNew['OBJID_MEM'] == memNew['ID_CEN_3']] = memNew[memNew['OBJID_MEM'] == 
                                                                        memNew['ID_CEN_3']]['P_CEN_3']
memNew['P_CEN_MEM'][memNew['OBJID_MEM'] == memNew['ID_CEN_4']] = memNew[memNew['OBJID_MEM'] == 
                                                                        memNew['ID_CEN_4']]['P_CEN_4']
memNew['P_CEN_MEM'][memNew['OBJID_MEM'] == memNew['ID_CEN_5']] = memNew[memNew['OBJID_MEM'] == 
                                                                        memNew['ID_CEN_5']]['P_CEN_5']

In [27]:
np.max(memNew['P_CEN_MEM'])

0.48453107

In [28]:
# Only select the ones with P_MEM >= 0.2 
memGood = memNew[memNew['P_MEM'] > 0.2]
len(memGood)

13584

In [29]:
# Update member ID 
for cc in clusters:
    for ii, index in enumerate(np.where(memGood['ID_1'] == cc)[0]):
        memGood['ID_MEM'][index] = str(cc).strip() + '_' + str(ii+1).strip()

In [30]:
# Save the output catalog 
outCat = catMem.replace('.fits', '_new.fits')
memGood.write(outCat, format='fits', overwrite=True)